# Image Comparison with Image Quality Assessment (IQA)

This notebook can be used to compare two images / volumes using different IQA metrics. The metrics used are:
- Peak Signal to Noise Ratio (PSNR)
- Root Mean Square Error (RMSE)
- Structured Similarity (SSIM)
- Multi-Scale Structural Similarity (MS-SSIM) [can only be used for 2D images]
- Feature Similarity Index (FSIM) [can only be used for 2D images]
- Visual Information Fidelity in *pixel* domain (VIFp) [can only be used for 2D images]
- Visual Saliency Index (VSI) [can only be used for 2D images]

**Important Notes:**
The calculated values for VIFp, SSIM, MS-SSIM are probably not correct in this implementation. Those values should be treated with caution.

**Usage:**
The metrics are implemented to calculate the scores for a 8 bit data range (0-255) per default. The data range can be changed by setting the `data_range` variable for each metric. Images are first loaded from .raw files or .mhd files and their corresponding .raw file, normalized to the chosen data range and then compared. The scores are then calculated and printed. The images are plotted to visually compare them. File names need to be given with the bit depth denoted as a suffix (e.g. _8bit.raw, _16bit.mhd) and the dimensions of the images need to be given in the file name (e.g. _512x512x512_). The images are assumed to be grayscale.

To use this notebook, first set the filepaths and parameters, the press 'Run All'.

**Requirements:**
The following packages have to be installed:
- numpy
- matplotlib
- pytorch
- piq
- scikit-image

## Import

In [26]:
import psnr, rmse, fsim, vif, ssim, msssim, vsi
from utils import load_data
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import savemat
import torch

## Data Loading and Setup

In [27]:
# Change these filepaths to your images (_r = reference, _m = modified)
file_path_img_r = '../../samples/Catec_Two_PlateIQI_20um/Catec_Two_PlateIQI_20um_1620proj_220kV_Rayscan-SimCT_800x800x1000_16bit.raw'
file_path_img_m = '../../samples/Catec_Two_PlateIQI_20um/Catec_Two_PlateIQI_20um_810proj_220kV_Rayscan-SimCT_800x800x1000_16bit.raw'

# Set these to true if you want to calculate the denominated metric for your data
calc_psnr = True
calc_rmse = True
calc_ssim = True
calc_msssim = True
calc_fsim = True
calc_vif = True
calc_vsi = True

In [28]:
# Data loading
img_r = load_data(file_path_img_r, data_range=255, normalize=True, batch=False)[0:1000, 200, 0:800]
img_m = load_data(file_path_img_m, data_range=255, normalize=True, batch=False)[0:1000, 200, 0:800]

In [29]:
img_r_tensor = torch.tensor(img_r).unsqueeze(0).unsqueeze(0)
img_m_tensor = torch.tensor(img_m).unsqueeze(0).unsqueeze(0)

In [30]:
savemat('img_m.mat', {'img_m': img_m})
savemat('img_r.mat', {'img_r': img_r})

## Calculation

### PSNR
Set the parameters for PSNR. The data range is set to 255 per default.

In [31]:
# Change this parameter
psnr_data_range = 255

In [32]:
if calc_psnr:
    metric_psnr_m = psnr.PSNR(data_range=psnr_data_range)
    metric_psnr_m.score(img_r, img_m)

### RMSE
The RMSE metric does not need any parameters.

In [33]:
if calc_rmse:
    metric_rmse_m = rmse.RMSE(data_range=255)
    metric_rmse_m.score(img_r, img_m)

### SSIM
Set the parameters for SSIM. The data range is set to 255 per default.

In [34]:
# Change this parameter
ssim_data_range = 255
ssim_calc_parameters = {'gaussian_weights': True, 'use_sample_covariance': False, 'sigma': 1.5}

In [35]:
if calc_ssim:
    metric_ssim_m = ssim.SSIM(data_range=ssim_data_range)
    metric_ssim_m.score(img_r, img_m,  **ssim_calc_parameters)

In [36]:
# sewar ssim
from sewar import ssim
print('sewar: ', ssim(img_r, img_m))

# piq ssim
from piq import ssim as piq_ssim
print('piq: ', piq_ssim(img_r_tensor, img_m_tensor, data_range=255))

# skimage ssim
from skimage.metrics import structural_similarity as skimage_ssim
print('skimage: ', skimage_ssim(img_r, img_m, data_range=255, gaussian_weights=True, use_sample_covariance=False, sigma=1.5))

# pytorch-msssim ssim
from pytorch_msssim import ssim as pytorch_msssim_ssim
print('pytorch_msssim: ', pytorch_msssim_ssim(img_r_tensor, img_m_tensor, data_range=255))

# pytorch-ignite ssim
from ignite.metrics import SSIM
metric = SSIM(data_range=255)
metric.update((img_r_tensor, img_m_tensor))
print('pytorch-ignite: ', metric.compute())

# xdesign ssim
from xdesign.metrics import ssim as xdesign_ssim
print('xdesign: ', xdesign_ssim(img_r, img_m, L=255)[1])

# IQA-pytorch ssim
from IQA_pytorch import SSIM
from torch.nn.functional import normalize
metric = SSIM()
print('IQA-pytorch: ', metric(normalize(img_r_tensor.float()), normalize(img_m_tensor.float()), as_loss=False))

sewar:  (0.6508314417296361, 0.6514188981938117)
piq:  tensor(0.8296)
skimage:  0.667623583012712
pytorch_msssim:  tensor(1.)
pytorch-ignite:  0.6743957127206505
xdesign:  0.7873974805075192
IQA-pytorch:  tensor([1.])


### MS-SSIM

Set the parameters for MS-SSIM. The data range is set to 255 per default.

In [37]:
# Change this parameter
msssim_data_range = 255

In [38]:
if calc_msssim:
    if len(img_r.shape) == 2:
        metric_msssim_m = msssim.MSSSIM(data_range=msssim_data_range)
        metric_msssim_m.score(img_r, img_m)
    else:
        print('MS-SSIM can only be used for 2D images')

In [39]:
# sewar msssim
from sewar import msssim
print('sewar: ', msssim(img_r, img_m))

# piq msssim
from piq import multi_scale_ssim as piq_msssim
print('piq: ', piq_msssim(img_r_tensor, img_m_tensor, data_range=255))

# pytorch-msssim msssim
try:
    from pytorch_msssim import ms_ssim as pytorch_msssim_msssim
    print('pytorch msssim: ', pytorch_msssim_msssim(img_r_tensor, img_m_tensor, data_range=255))
except:
    pass

# xdesign msssim
from xdesign.metrics import msssim as xdesign_msssim
print('xdesign: ', xdesign_msssim(img_r, img_m, L=255)[1])

# IQA-pytorch msssim
from IQA_pytorch import MS_SSIM
from torch.nn.functional import normalize
metric = MS_SSIM()
print('IQA_pytorch: ', metric(normalize(img_r_tensor.float()), normalize(img_m_tensor.float()), as_loss=False))

sewar:  (0.8631333216904347+0j)
piq:  tensor(0.8630)
xdesign:  0.5962385859460634
IQA_pytorch:  tensor([1.])


### FSIM
Set the parameters for FSIM. The data range is set to 255 per default. The chromatic parameter can be set to true if the images are RGB images.

In [40]:
# Change these parameters
fsim_data_range = 255
fsim_load_parameters = {'chromatic': False}

In [41]:
if calc_fsim:
    if len(img_r.shape) == 2:
        metric_fsim_m = fsim.FSIM(data_range=fsim_data_range, **fsim_load_parameters)
        metric_fsim_m.score(img_r, img_m)
    else:
        print('FSIM can only be used for 2D images')

### VIFp
Set the parameters for VIFp. The data range is set to 255 per default. The sigma_n_sq parameter can be set to adapt the metric for the variance of visual noise. This parameter is set to 3.2 per default and has to be changed according to the data. Currently there is no recommendation how to set this parameter.

In [51]:
# Change these parameters
vif_data_range = 255
vif_load_parameters = {'chromatic': False}
vif_calc_parameters = {'sigma_n_sq': 2}

In [52]:
if calc_vif:
    if len(img_r.shape) == 2:
        metric_vifp_m = vif.VIFp(data_range=255, **vif_load_parameters)
        metric_vifp_m.score(img_r, img_m, **vif_calc_parameters)
    else:
        raise Exception('VIFp can only be used for 2D images')

### VSI

Set the parameters for VSI. The data range is set to 255 per default.

In [44]:
# Change these parameters
vsi_data_range = 255
vsi_load_parameters = {'chromatic': False}

In [45]:
if calc_vsi:
    if len(img_r.shape) == 2:
        metric_vsi_m = vsi.VSI(data_range=vsi_data_range, **vsi_load_parameters)
        metric_vsi_m.score(img_r, img_m)
    else:
        raise Exception('VSI can only be used for 2D images')

In [46]:
# Set the number of decimals to be printed
decimals = 4

In [53]:
if calc_psnr:
    metric_psnr_m.print_score(decimals)
if calc_rmse:
    metric_rmse_m.print_score(decimals)
if calc_ssim:
    metric_ssim_m.print_score(decimals)
if calc_msssim:
    metric_msssim_m.print_score(decimals)
if calc_fsim:
    metric_fsim_m.print_score(decimals)
if calc_vif:
    metric_vifp_m.print_score(decimals)
if calc_vsi:
    metric_vsi_m.print_score(decimals)

PSNR: 29.7071
RMSE: 8.3404
SSIM: 0.6676
MS-SSIM: 0.863
FSIM: 0.9479
VIFp: 0.1005
VSI: 0.9919


## Plotting

In [48]:
print('Shape of reference image:', img_r.shape)

Shape of reference image: (1000, 800)


In [49]:
# Set the area to be plotted (two dimensions have to be given as range, the third value is the slice to be plotted)
x = None
x_1 = 0
x_2 = 1000

y = 200
y_1 = None
y_2 = None

z = None
z_1 = 0
z_2 = 800

In [50]:
if x is not None:
    img_r_plot = img_r[x, y_1:y_2, z_1:z_2]
    img_m_plot = img_m[x, y_1:y_2, z_1:z_2]
elif y is not None:
    img_r_plot = img_r[x_1:x_2, y, z_1:z_2]
    img_m_plot = img_m[x_1:x_2, y, z_1:z_2]
elif z is not None:
    img_r_plot = img_r[x_1:x_2, y_1:y_2, z]
    img_m_plot = img_m[x_1:x_2, y_1:y_2, z]
else:
    raise Exception('Area to be plotted was not correctly specified')

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [ ]:
fig, axs = plt.subplots(1,2, dpi=300)
axs[0].imshow(img_r_plot, cmap="gray")
axs[1].imshow(img_m_plot, cmap="gray")

fig.suptitle('Image Comparison and IQA metric values', y=0.92)
axs[0].set_title('Reference image')
axs[1].set_title('Modified image')

if calc_psnr:
    fig.text(0.4, 0.09, f'RMSE: {metric_rmse_m.score_val:.2f}', ha='center', fontsize=8)
if calc_rmse:
    fig.text(0.4, 0.06, f'PSNR: {metric_psnr_m.score_val:.2f} dB', ha='center', fontsize=8)
if calc_ssim:
    fig.text(0.4, 0.03, f'SSIM: {metric_ssim_m.score_val:.2f}', ha='center', fontsize=8)
if calc_msssim:
    fig.text(0.4, 0.00, f'MS-SSIM: {metric_msssim_m.score_val:.2f}', ha='center', fontsize=8)
if calc_fsim:
    fig.text(0.6, 0.09, f'FSIM: {metric_fsim_m.score_val:.2f}', ha='center', fontsize=8)
if calc_vif:
    fig.text(0.6, 0.06, f'VIFp: {metric_vifp_m.score_val:.2f}', ha='center', fontsize=8)
if calc_vsi:
    fig.text(0.6, 0.03, f'VSI: {metric_vsi_m.score_val:.2f}', ha='center', fontsize=8)
    
axs[0].axis('off')
axs[1].axis('off');

## Conclusion

| Metric | Code OK | Value OK |
| --- |---------|----------|
| PSNR | OK      | OK       |
| RMSE | OK      | OK       |
| VIFp | OK      | *NOT* OK |
| FSIM | OK      | ?        |
| SSIM | OK      | OK       |
| MSSSIM | OK      | OK       |
| VSI | OK      | OK       |
| MAD | --      | --       |
| GSM | --      | --       |
| SFF | --      | --       |
| CNR | --      | --       |
| Ma | --      | --       |
| PI | --      | --       |
| NIQE | --      | --       |